In [1]:
from bs4 import BeautifulSoup
import random
import html
import requests
import re
from time import sleep
from urllib.parse import quote
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

In [13]:
site = 'https://myshows.me/community/users/?page={}'
max_num_page = 5000

In [14]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)
    
def extract_users_from_page(site_num):
    soup = BeautifulSoup(requests.get(site.format(site_num)).text, 'html.parser')
    res = []
    
    links = soup.find_all('a', class_='userBlock linkBlock')
    for link in links:
        res.append(link.attrs['href'])
    
    return res

def extract_grades_from_user_page(user_url):
    soup = BeautifulSoup(requests.get(user_url).text, 'html.parser')
    res = {}
    
    tabs = soup.find_all('div', class_='tabs_cont')
    for i in [0, 1, ]
    completed_tab = tabs[-1]
    all_grades = completed_tab.find_all('tr')
    for grade in all_grades[:-1]:
        grade_parts = grade.find_all('td')
        title = grade_parts[0].find('a').text
        grade_value = grade_parts[1].find('span').attrs['class'][1][1]
        res[title] = grade_value
    
    return res

In [15]:
%%time


with ProcessPoolExecutor(max_workers=5) as executor:
    result = list(executor.map(extract_users_from_page,
                               list(range(max_num_page))))
    
unpack_result = []
for l in result:
    unpack_result += l

with open('user_urls.txt', 'w') as fin:
    for row in unpack_result:
        fin.write(row + '\n')

CPU times: user 6.7 s, sys: 1.15 s, total: 7.85 s
Wall time: 22min 40s


In [18]:
print(len(set(unpack_result)), len(unpack_result))
user_urls = list(set(unpack_result))

149981 150000


In [17]:
extract_grades_from_user_page('https://myshows.me/Vlad_Sherbi')

{'Время приключений': '5',
 'Блудливая Калифорния': '5',
 'Декстер': '5',
 'Друзья': '5',
 'Гравити Фолз': '5',
 'Доктор Хаус': '5',
 'Как я встретил вашу маму': '5',
 'Остаться в живых': '5',
 'Марко Поло': '5',
 'Защитники': '5',
 'Клиника': '5',
 'Человек-Паук': '5',
 'Дневники вампира': '5',
 'Секретные материалы': '5',
 'Черные паруса': '4',
 'Во все тяжкие': '4',
 'Константин': '4',
 'Вспомни, что будет': '4',
 'Вечность': '4',
 'Футурама': '4',
 'Гримм': '4',
 'Ганнибал': '4',
 'Обмани меня': '4',
 'Области тьмы': '4',
 'Агент Картер': '4',
 'Железный кулак': '4',
 'Люк Кейдж': '4',
 'Отбросы': '4',
 'Восьмое чувство': '4',
 'Волчонок': '4',
 'Древние': '4',
 'Кровавая гонка\t': '3',
 'Герои': '3',
 'Сонная Лощина': '3',
 'Офис': '3',
 'Под куполом': '3',
 'Сверхлюди': '2'}